<a href="https://colab.research.google.com/github/nvakiet/SparkML-Regression_And_Classification/blob/main/SparkML_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Spark on Google Colab

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

!tar xf spark-3.2.1-bin-hadoop3.2.tgz

!pip install -q findspark
!pip install pyspark

^C


# Download the "Predict Future Sales" dataset from Kaggle

In [ ]:
# Install Kaggle python package and create directory for kaggle credential
!pip install kaggle
!mkdir ~/.kaggle
# Copy the Kaggle credential to runtime machine (remember to mount the Google Drive containing the credential file)
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

# Download the dataset and unzip
!kaggle competitions download -c competitive-data-science-predict-future-sales
!unzip competitive-data-science-predict-future-sales.zip


ERROR: Operation cancelled by user
mkdir: cannot create directory ‘/root/.kaggle’: File exists
competitive-data-science-predict-future-sales.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  competitive-data-science-predict-future-sales.zip
replace item_categories.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Import packages and initialize a Spark session

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

from pyspark.sql import functions as f
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.ml.evaluation import *

In [ ]:
spark = (SparkSession
        .builder
        .appName('Lab 3 - SparkML: Regression')
        .getOrCreate()
)

# Load train and test datasets into Pyspark

In [ ]:
train = spark.read.csv("sales_train.csv", header = True, inferSchema = True)
test = spark.read.csv("test.csv", header=True, inferSchema=True)

# Raw data overview
Firstly, let's see what does the raw data look like.

In [ ]:
# Print the dataframe schema
print("Training data schema")
train.printSchema()
print("Test data schema")
test.printSchema()

In [ ]:
# See the first 10 samples
train.show(10)

Let's generate a summary of the whole dataframe to have a better understanding of the problem

In [ ]:
print("Column number:", len(train.columns))
print("Raw data summary:")
train.summary().show()

From the summary, we can see the dataset has a shape of (2935849,6). This is a time series where the item price and sales count depend on the dates. There's also the fact that the problem requires us to predict the sales count by month while this dataset is indexed by date, so some aggregation must be done on this dataframe.  
Judging from the summary (mean, std, min, max) of item_price and item_cnt_day, there're definitely outliers in the data, and maybe some missing values.  
Because the basic Spark MLlib library doesn't have any class for time series modeling, we must perform some feature engineering to convert this dataset from a time series forecasting problem to a regression problem so we can fit it with SparkML's regressors.

# Data preprocessing
To prevent data leakage problem, we remove any item in training set which doesn't exist in test set. With Spark, this can be done by doing a right outer join.

In [ ]:
# Perform right outer join
train = (train.join(test, [train.shop_id == test.shop_id, train.item_id == test.item_id], "rightouter")
              .select(train.columns))
# See if the number of rows decreases
train.count()

# Exploratory Data Analysis (EDA)